In [6]:
import pandas as pd
pd.options.display.max_columns = 99
crunchbase = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

In [8]:
mv_list = []
for chunk in crunchbase:
    mv_list.append(chunk.isnull().sum())
    
combined_mv_vc = pd.concat(mv_list)
unique_combined_mv_vc = combined_mv_vc.groupby(combined_mv_vc.index).sum()
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [10]:
crunchbase = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
counter = 0
series_memory_fp = pd.Series()
for chunk in crunchbase:
    if counter == 0:
        series_memory_fp = chunk.memory_usage(deep=True)
    else:
        series_memory_fp += chunk.memory_usage(deep=True)
    counter += 1

# Drop memory footprint calculation for the index.
series_memory_fp = series_memory_fp.drop('Index')
series_memory_fp

company_permalink         4057788
company_name              3591326
company_category_code     3421104
company_country_code      3172176
company_state_code        3106051
company_region            3411585
company_city              3505926
investor_permalink        4980548
investor_name             3915666
investor_category_code     622424
investor_country_code     2647292
investor_state_code       2476607
investor_region           3396281
investor_city             2885083
funding_round_type        3410707
funded_at                 3542185
funded_month              3383584
funded_quarter            3383584
funded_year                422960
raised_amount_usd          422960
dtype: int64

In [12]:
series_memory_fp.sum() / (1024 * 1024)

56.9876070022583

In [13]:
unique_combined_mv_vc.sort_values()

company_country_code          1
company_name                  1
company_permalink             1
company_region                1
investor_region               2
investor_permalink            2
investor_name                 2
funded_quarter                3
funded_at                     3
funded_month                  3
funded_year                   3
funding_round_type            3
company_state_code          492
company_city                533
company_category_code       643
raised_amount_usd          3599
investor_country_code     12001
investor_city             12480
investor_state_code       16809
investor_category_code    50427
dtype: int64

In [14]:
drop_cols = ['investor_permalink', 'company_permalink', 'investor_category_code']
keep_cols = chunk.columns.drop(drop_cols)

In [15]:
keep_cols.tolist

<bound method Index.tolist of Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')>

In [16]:
col_types = {}
crunchbase = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

for chunk in crunchbase:
    for col in chunk.columns:
        if col not in col_types:
            col_types[col] = [str(chunk.dtypes[col])]
        else:
            col_types[col].append(str(chunk.dtypes[col]))

In [17]:
uniq_col_types = {}
for k,v in col_types.items():
    uniq_col_types[k] = set(col_types[k])
uniq_col_types

{'company_category_code': {'object'},
 'company_city': {'object'},
 'company_country_code': {'object'},
 'company_name': {'object'},
 'company_region': {'object'},
 'company_state_code': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'funding_round_type': {'object'},
 'investor_city': {'float64', 'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_name': {'object'},
 'investor_region': {'object'},
 'investor_state_code': {'float64', 'object'},
 'raised_amount_usd': {'float64'}}

In [19]:
chunk.head()

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
50000,NuORDER,fashion,USA,CA,Los Angeles,West Hollywood,Mortimer Singer,NaN,NaN,unknown,NaN,series-a,2012-10-01,2012-10,2012-Q4,2012,3060000.0
50001,ChaCha,advertising,USA,IN,Indianapolis,Carmel,Morton Meyerson,NaN,NaN,unknown,NaN,series-b,2007-10-01,2007-10,2007-Q4,2007,12000000.0
50002,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2008-04-18,2008-04,2008-Q2,2008,500000.0
50003,Binfire,software,USA,FL,Bocat Raton,Bocat Raton,Moshe Ariel,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,750000.0
50004,Unified Color,software,USA,CA,SF Bay,South San Frnacisco,Mr. Andrew Oung,NaN,NaN,unknown,NaN,angel,2010-01-01,2010-01,2010-Q1,2010,NaN


In [20]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
crunchbase = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')

for chunk in crunchbase:
    chunk.to_sql("investments", conn, if_exists='append', index=False)

In [22]:
!wc -c crunchbase.db

10768384 crunchbase.db


In [23]:
print("Total MB for Crunchbase.db:", 10768384 / 1024**2)

Total MB for Crunchbase.db: 10.26953125


In [24]:
crunch_df = pd.read_sql('select company_category_code, investor_name, raised_amount_usd, funding_round_type from investments;', conn)
print(crunch_df.head())

  company_category_code      investor_name  raised_amount_usd  \
0           advertising  1-800-FLOWERS.COM          2000000.0   
1                  news        10Xelerator            20000.0   
2             messaging        10Xelerator            20000.0   
3              software        10Xelerator            20000.0   
4                   web        10Xelerator            20000.0   

  funding_round_type  
0           series-a  
1              other  
2              other  
3              angel  
4              other  


In [25]:
cmp_type_df = crunch_df.groupby(crunch_df.company_category_code).sum()
cmp_type_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 

print(cmp_type_df.head())

                       raised_amount_usd
company_category_code                   
biotech                     1.103964e+11
software                    7.308452e+10
mobile                      6.477738e+10
cleantech                   5.270523e+10
enterprise                  4.586093e+10


In [26]:
investor_name_df = crunch_df.groupby(crunch_df.investor_name).sum()
investor_name_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 

print(investor_name_df.head())

                                  raised_amount_usd
investor_name                                      
Kleiner Perkins Caufield & Byers       1.121783e+10
New Enterprise Associates              9.692542e+09
Accel Partners                         6.472126e+09
Goldman Sachs                          6.375459e+09
Sequoia Capital                        6.039402e+09


In [27]:
investor_name_counts = crunch_df.investor_name.value_counts()

investor_name_df["count"] = investor_name_counts
investor_name_df["avg investment"] = investor_name_df['raised_amount_usd'] / investor_name_df["count"] 
investor_name_df.sort_values('raised_amount_usd', ascending=False, inplace=True) 
print(investor_name_df.head())

                                  raised_amount_usd  count  avg investment
investor_name                                                             
Kleiner Perkins Caufield & Byers       1.121783e+10    393    2.854409e+07
New Enterprise Associates              9.692542e+09    445    2.178099e+07
Accel Partners                         6.472126e+09    322    2.009977e+07
Goldman Sachs                          6.375459e+09    123    5.183300e+07
Sequoia Capital                        6.039402e+09    369    1.636694e+07


In [30]:
funding_round_type_counts = crunch_df.funding_round_type.value_counts()
funding_round_type_df = pd.DataFrame()
funding_round_type_df['count'] = funding_round_type_counts
funding_round_type_df.sort_values('count', ascending=False, inplace=True) 

print(funding_round_type_df.head())

print(funding_round_type_df.tail())

           count
series-a   13938
series-c+  10870
angel       8989
venture     8917
series-b    8794
                count
series-b         8794
other             964
private-equity    357
post-ipo           33
crowdfunding        5
